# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import sys
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML, display
%matplotlib inline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [2]:

def grayscale(image):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return  cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)



## Custom Helper Functions

In [11]:
def create_dir(dir):
    """
    Create a directoy with the given name, if it does not exists.
    """
    if not os.path.exists (dir):
        os.makedirs(dir)
        
def get_area_of_interest_boundaries (image):
    """
    Takes an image and returns the boundaries for area of interest polygon
    """
    # top y cordinate of the polygon  starts at 60 % in this case
    top_y = int(image.shape[0] * 0.6)
    
    #top x-es are 2.5% each side from the center
    top_x1 = int ((image.shape[1]/2) - (image.shape[1] * 0.025))
    top_x2 = int((image.shape[1]/2) + (image.shape[1] * 0.025))
    
    return top_y, top_x1, top_x2

def get_area_of_interest (image, area='full'):
    """
    This function is used to return the polygon for area of interest.
    image: use to get the shape of the input image
    area : full  -  Get the full area of interest
           left  -  Get the left half of area of interest
           right -  Get  the right half of area of interest 
    """
    # let's get the section of interest.
    top_y, top_x1, top_x2 = get_area_of_interest_boundaries (image)
    section_of_interest = None
   
    if area == 'left':
        # left half of the area of interest

        section_of_interest = np.array([[(0,image.shape[0]),
                                         (top_x1, top_y), 
                                         (int(image.shape[1]/2), top_y), 
                                         (int(image.shape[1]/2),image.shape[0])]],
                                       dtype=np.int32)
    elif area == 'right':
        # right half of the area of interest

        section_of_interest = np.array([[ (int(image.shape[1]/2),image.shape[0]),
                                 (int(image.shape[1]/2), top_y), 
                                 (top_x2, top_y), 
                                 (image.shape[1],image.shape[0])]],
                               dtype=np.int32)

    else:

        section_of_interest = np.array([[(0,image.shape[0]),
                                 (top_x1, top_y), 
                                 (top_x2, top_y), 
                                 (image.shape[1],image.shape[0])]],
                               dtype=np.int32)

            
    return section_of_interest


# take in a line and return its slope
# Note: can be improved to hadle infinite slope.
def slope(line):
    """
    Returns the slope of a line.
    """
    for x1,y1,x2,y2 in line:
        if (y1 == y2):
            # horizondal and vertical lines are ignored.
            return 0
        return (y2-y1)/(x2-x1)


def fit_lane_lines (img, out_image):
    """
    Extrapolates lane lines from img and draws them out_image
    img (in)           : input image
    out_image (in/out) : output image
    """
    rows,cols = img.shape[:2]

    # Find all pixels in the line and do a fit line.
    pixels = cv2.findNonZero(img)
    
    if pixels is not None:
        [vx,vy,x,y] = cv2.fitLine( pixels, cv2.DIST_L2,0,0.01,0.01)
        
        # The line should fit with in the area of interest
        # here calculate the boudaries of the line, with in the area of interest
        top_y, top_x1, top_x2 = get_area_of_interest_boundaries (img)
        leftx = int( ((top_y-y) * vx/vy) + x )
        rightx = int( ((rows-y) * vx/vy) + x )

        cv2.line(out_image, (leftx, top_y), (rightx, rows-1), [255, 0, 0], 8)
        

def draw_full_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    Take in an image and the lines given by hough transform.
    Draw one full line for each lane
    
    img: Image to draw the lines on
    lines: Array of lines give by hough transform
    """
    left_lane = []
    right_lane = []
    final_lines = []
    
    for line in lines:

        # filter off, parallel line like front of the car of start of the bridge
        # in challenge video
        if -0.3 <= slope(line) <= 0  or  0 <= slope(line) <= 0.3 :
            continue
        
        # Here sort the lines to be left or right lane candidates.            
        if slope(line) < 0:
            # left lane
            left_lane.append(line)
        else:
            # right lane
            right_lane.append(line)

    # One image for left, right and out.
    # The idea here, is to plot one lane segments on an image at a time,
    # and then do a fitlane to draw a full line.
    left_image = np.zeros_like(img)  
    right_image = np.zeros_like(img)  
    out_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)  
    
    # Draw one lane at a time.
    draw_lines (left_image, left_lane)
    draw_lines (right_image, right_lane)

    # Do the fitlanes of the output image
    fit_lane_lines ( region_of_interest (left_image, get_area_of_interest(left_image, 'left')), out_image)
    fit_lane_lines ( region_of_interest (right_image, get_area_of_interest(right_image, 'right')), out_image)

    return (out_image) 


def hough_lines_full(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    Draws a full line after extraploating lane segments.
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = draw_full_lines(np.zeros_like(img), lines)
    return line_img

def display_output_images(dir):
    """
    Display all the images from the given directory
    """
    html_div = "<div>"
    for file in os.listdir(dir):
        image = dir + "/" + file 
        html_div += '<img src="' + image + '"' + """width="300" height="110" style="float:left; margin:1px"/> """
    html_div += "</div>"
    
    display (HTML(html_div))
    

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [12]:
import os

os.listdir("test_images/")


['solidYellowCurve2.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg',
 'solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'solidWhiteRight.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [13]:
def detect_lanes_in_image (image):
    
    gray = grayscale(image)

    # blur for softening unwanted edges
    gray = gaussian_blur (gray, 5)

    # canny for detecting the edges
    gray = canny(gray, 50, 150)
    
    # mask everthing but region of interest
    gray = region_of_interest (gray, get_area_of_interest(image))

    # hough transform to find lines
    line_image = hough_lines_full (gray, 2, np.pi/180, 15, 40, 20)

    # output image, add the lines on top of original image
    out_image = weighted_img (line_image, image,  β=1.)
    
    return out_image



# Image tests

In [14]:
create_dir ("test_images_output")
create_dir ("test_videos_output")

for file in os.listdir("test_images"):
    image = mpimg.imread("test_images/" + file )
    out_image = detect_lanes_in_image (image)
    cv2.imwrite('test_images_output/out_' + file , cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB))
display_output_images ("test_images_output")

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [15]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = detect_lanes_in_image(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [16]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(8,9)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:09<00:00, 22.40it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 3.77 s, sys: 251 ms, total: 4.02 s
Wall time: 11.2 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [18]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(24,26)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:33<00:00, 20.48it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 11.9 s, sys: 719 ms, total: 12.6 s
Wall time: 34.6 s


In [19]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [20]:
def filter_yellow_and_white (image):
    """
    Filters out yellow and white colors from an image.
    """
    #Boundaries for white
    white_low = np.array([0,0,200])
    white_high = np.array([180,255,255])
    
    #Boundaries for yellow
    yellow_low = np.array([20,100,100])
    yellow_high = np.array([30,255,255])
    
    #convert image to HSV
    hsv = cv2.cvtColor (image, cv2.COLOR_BGR2HSV)
    
    #find each mask
    mask = cv2.inRange(hsv, white_low, white_high)
    mask |= cv2.inRange(hsv, yellow_low, yellow_high)
    
    # use the mask find just the lane image
    lane_image = cv2.bitwise_and (image, image, mask = mask)
    
    return lane_image
    
def grayscale_after_filtering_lanes (image):
    """Applies the Grayscale transform on an image after 
    Filtering out yellow and white lanes
    """
    lane_image = filter_yellow_and_white (image)
    return  cv2.cvtColor(lane_image, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def detect_lanes_in_image_challenge (image):
    """
    The same pipline but, filter out the lane colors
    before converting to gray.
    
    Note:
    Putting this as a seperate function, as this is affecting the quality of
    original videos.
    Bit more averaging logic is needed to make this fully functional for all
    videos.
    """
    
    gray = grayscale_after_filtering_lanes(image)

    # let's do the blur
    gray = gaussian_blur (gray, 5)
    # canny
    gray = canny(gray, 50, 150)
    
    # region of interest
    gray = region_of_interest (gray, get_area_of_interest(image))

    #hough transform
    line_image = hough_lines_full (gray, 2, np.pi/180, 15, 40, 20)

    #output image
    out_image = weighted_img (line_image, image,  β=1.)
    
    return out_image    

def process_image2(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = detect_lanes_in_image_challenge (image)
    return result

In [21]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(4,6)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(detect_lanes_in_image_challenge)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/challenge.mp4
[MoviePy] Writing video test_videos_output/challenge.mp4


100%|██████████| 251/251 [00:26<00:00,  9.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge.mp4 

CPU times: user 12.6 s, sys: 965 ms, total: 13.5 s
Wall time: 29.6 s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))